In [1]:
from itertools import product
import numpy as np
import pandas as pd
from ADRprofilePrediction import Pairs2Mat, evaluation
from Models import loadHyperpar
import seaborn as sns 
import matplotlib.pylab as plt
from matplotlib.ticker import MultipleLocator
import matplotlib.patheffects as path_effects
import json
import pingouin as pg

In [2]:
import sklearn
print(sklearn.__version__)

0.24.2


In [3]:
features_dict = {
    "target":Pairs2Mat(path="data/drug_target.tsv",colname1="0",colname2="1"),
    "enzyme":Pairs2Mat(path="data/drug_enzyme.tsv",colname1="0",colname2="1"),
    "Chem":pd.read_csv("data/drug_chemsfp.tsv",sep = "\t",header=0,index_col=0),
    "DGI":Pairs2Mat(path="data/interactions.tsv",colname1="drug_claim_name",colname2="gene_name"),
    "transporter":Pairs2Mat(path="data/drug_transporter.tsv",colname1="0",colname2="1"),
    "pathway":Pairs2Mat(path="data/drug_pathway.tsv",colname1="0",colname2="1"),
    "indication":Pairs2Mat(path="data/drug_indication.tsv",colname1="1_x",colname2="6")
}


In [4]:
filter = "all"
SEs = {}
if filter == "all":
    SIDER = Pairs2Mat(path="data/drug_se.tsv",colname1="1_x",colname2="5")
    column_sums = np.sum(SIDER, axis=0)
    SEs["SIDER"] = SIDER.loc[:, (column_sums >= 5)]

    OFFSIDERS = Pairs2Mat(path="data/OFFSIDES.csv",colname1="drug_concept_name",colname2="condition_concept_name",sep = ",")
    column_sums = np.sum(OFFSIDERS, axis=0)
    SEs["OFFSIDES"] = OFFSIDERS.loc[:, column_sums >= 5]
elif filter == "rare":
    SIDER = Pairs2Mat(path="data/drug_se.tsv",colname1="1_x",colname2="5")
    column_sums = np.sum(SIDER, axis=0)
    SEs["SIDER"] = SIDER.loc[:, (column_sums < 50)]

    OFFSIDERS = Pairs2Mat(path="data/OFFSIDES.csv",colname1="drug_concept_name",colname2="condition_concept_name",sep = ",")
    column_sums = np.sum(OFFSIDERS, axis=0)
    SEs["OFFSIDES"] = OFFSIDERS.loc[:, column_sums < 50]


In [5]:
SEs["SIDER"].shape

(1344, 2556)

In [6]:
SEs["OFFSIDES"].shape

(2730, 12750)

In [7]:
features_names = ["target", "enzyme", "Chem", "DGI", "transporter", "pathway", "indication"]
# SEs_names = ["SIDER", "OFFSIDES"]
# methods = ["SKR", "KR", "KRR", "Naive", "LNSM_RLN", "LNSM_jaccard", "VKR"]
methods = ["SKR", "KRR", "VKR", "Naive", "LNSM_RLN", "LNSM_jaccard"]
# methods = ["SKR", "KR", "KRR", "Naive", "LNSM_RLN", "LNSM_jaccard", "VKR", "SVM", "OCCA", "SCCA", "RF", "BRF"]
tuning_metrices=["AUROC", "AUPR", "AUROCperdrug", "AUPRperdrug"]
metrice_names = ["AUPR+AUROC", "AUPR+AUROCperdrug", "AUROC", "AUPR", "AUROCperdrug", "AUPRperdrug"]

In [8]:
A = 10**np.arange(-2, 3, 1, dtype=float)
B = np.arange(0.1, 1, 0.1, dtype=float)
C = np.arange(5, 20, 5, dtype=int)
A10 = 10**np.arange(1, 2, 1, dtype=float)
A100 = 10**np.arange(2, 3, 1, dtype=float)
all_hyperparlist = {
    "SKR":[A,B,A10,A100], 
    # "KR":[A,A], 
    "KRR":[A,A],
    "VKR":[A,A,C], 
    "Naive":[], 
    "LNSM_RLN":[B,A], 
    "LNSM_jaccard":[B], 
    # "SVM":[A,A,A], 
    # "OCCA":[], 
    # "SCCA":[A], 
    # "RF":[C], 
    # "BRF":[C]
}

In [9]:
SEs_name = "OFFSIDES"
metrice = "AUPR"

In [10]:
hyperpars = {}
hyperpars["nested_cv"] = {}
hyperpars["cv"] = {}
hyperparsOut = {}
hyperparsOut["nested_cv"] = {}
hyperparsOut["cv"] = {}
results = {}
results["nested_cv"] = {}
results["cv"] = {}

In [11]:
hyperpars["nested_cv"]["SKR"] = {}
hyperpars["nested_cv"]["SKR"]["target"] = [
    (0.01, 0.6, 10, 100),
    (0.01, 0.6, 10, 100),
    (0.01, 0.6, 10, 100),
    (0.01, 0.6, 10, 100),
    (0.01, 0.5, 10, 100),
    ]
hyperpars["nested_cv"]["SKR"]["enzyme"] = [
    (1, 0.9, 10, 100),
    (1, 0.1, 10, 100),
    (1, 0.1, 10, 100),
    (1, 0.1, 10, 100),
    (1, 0.1, 10, 100),
    ]
hyperpars["nested_cv"]["SKR"]["Chem"] = [
    (1, 0.6, 10, 100),
    (1, 0.5, 10, 100),
    (1, 0.5, 10, 100),
    (10, 0.2, 10, 100),
    (10, 0.1, 10, 100),
    ]
hyperpars["nested_cv"]["SKR"]["DGI"] = [
    (0.1, 0.1, 10, 100),
    (0.1, 0.1, 10, 100),
    (0.1, 0.1, 10, 100),
    (0.1, 0.1, 10, 100),
    (0.1, 0.1, 10, 100),
    ]
hyperpars["nested_cv"]["SKR"]["transporter"] = [
    (0.01, 0.6, 10, 100),
    (0.01, 0.6, 10, 100),
    (0.01, 0.6, 10, 100),
    (0.01, 0.6, 10, 100),
    (0.01, 0.6, 10, 100),
    ]
hyperpars["nested_cv"]["SKR"]["pathway"] = [
    (0.01, 0.4, 10, 100),
    (0.01, 0.4, 10, 100),
    (0.01, 0.3, 10, 100),
    (0.01, 0.4, 10, 100),
    (0.01, 0.4, 10, 100),
    ]
hyperpars["nested_cv"]["SKR"]["indication"] = [
    (0.1, 0.1, 10, 100),
    (0.1, 0.1, 10, 100),
    (0.1, 0.1, 10, 100),
    (0.1, 0.1, 10, 100),
    (0.1, 0.1, 10, 100),
    ]
hyperpars["cv"]["SKR"] = {}
hyperpars["cv"]["SKR"]["target"] = (100, 0.1, 10, 100)
hyperpars["cv"]["SKR"]["enzyme"] = (1, 0.9, 10, 100)
hyperpars["cv"]["SKR"]["Chem"] = (1, 0.9, 10, 100)
hyperpars["cv"]["SKR"]["DGI"] =(0.1, 0.9, 10, 100)
hyperpars["cv"]["SKR"]["transporter"] = (0.01, 0.9, 10, 100)
hyperpars["cv"]["SKR"]["pathway"] = (100, 0.9, 10, 100)
hyperpars["cv"]["SKR"]["indication"] = (0.1, 0.9, 10, 100)


In [12]:
hyperpars["nested_cv"]["KRR"] = {}
hyperpars["nested_cv"]["KRR"]["target"] = [
    (1, 10),
    (1, 10),
    (1, 10),
    (1, 10),
    (0.1, 10),
    ]
hyperpars["nested_cv"]["KRR"]["enzyme"] = [
    (1, 10),
    (1, 10),
    (1, 10),
    (1, 10),
    (1, 10),
    ]
hyperpars["nested_cv"]["KRR"]["Chem"] = [
    (10, 10),
    (10, 10),
    (10, 10),
    (10, 10),
    (10, 10),
    ]
hyperpars["nested_cv"]["KRR"]["DGI"] = [
    (0.1, 10),
    (0.1, 10),
    (0.1, 10),
    (0.1, 10),
    (0.1, 10),
    ]
hyperpars["nested_cv"]["KRR"]["transporter"] = [
    (0.1, 10),
    (1, 10),
    (1, 10),
    (0.01, 10),
    (0.1, 10),
    ]
hyperpars["nested_cv"]["KRR"]["pathway"] = [
    (0.1, 10),
    (0.01, 10),
    (0.1, 10),
    (0.01, 10),
    (0.1, 10),
    ]
hyperpars["nested_cv"]["KRR"]["indication"] = [
    (0.1, 10),
    (0.1, 10),
    (0.1, 10),
    (0.1, 10),
    (0.1, 10),
    ]
hyperpars["cv"]["KRR"] = {}
hyperpars["cv"]["KRR"]["target"] = (100, 10)
hyperpars["cv"]["KRR"]["enzyme"] = (100, 10)
hyperpars["cv"]["KRR"]["Chem"] = (0.1, 100)
hyperpars["cv"]["KRR"]["DGI"] = (0.1, 100)
hyperpars["cv"]["KRR"]["transporter"] = (0.01, 100)
hyperpars["cv"]["KRR"]["pathway"] = (100, 100)
hyperpars["cv"]["KRR"]["indication"] = (1, 100)

In [13]:
hyperpars["nested_cv"]["Naive"] = {}
hyperpars["nested_cv"]["Naive"]["target"] = [
    (),
    (),
    (),
    (),
    (),
    ]
hyperpars["nested_cv"]["Naive"]["enzyme"] = [
    (),
    (),
    (),
    (),
    (),
    ]
hyperpars["nested_cv"]["Naive"]["Chem"] = [
    (),
    (),
    (),
    (),
    (),
    ]
hyperpars["nested_cv"]["Naive"]["DGI"] = [
    (),
    (),
    (),
    (),
    (),
    ]
hyperpars["nested_cv"]["Naive"]["transporter"] = [
    (),
    (),
    (),
    (),
    (),
    ]
hyperpars["nested_cv"]["Naive"]["pathway"] = [
    (),
    (),
    (),
    (),
    (),
    ]
hyperpars["nested_cv"]["Naive"]["indication"] = [
    (),
    (),
    (),
    (),
    (),
    ]
hyperpars["cv"]["Naive"] = {}
hyperpars["cv"]["Naive"]["target"] = ()
hyperpars["cv"]["Naive"]["enzyme"] = ()
hyperpars["cv"]["Naive"]["Chem"] =()
hyperpars["cv"]["Naive"]["DGI"] = ()
hyperpars["cv"]["Naive"]["transporter"] = ()
hyperpars["cv"]["Naive"]["pathway"] = ()
hyperpars["cv"]["Naive"]["indication"] = ()

In [14]:
hyperpars["nested_cv"]["VKR"] = {}
hyperpars["nested_cv"]["VKR"]["target"] = [
    (10, 0.1, 15),
    (10, 0.01, 15),
    (10, 0.1, 15),
    (10, 0.1, 15),
    (10, 0.01, 15),
    ]
hyperpars["nested_cv"]["VKR"]["enzyme"] = [
    (10, 1, 15),
    (10, 0.01, 5),
    (10, 0.01, 10),
    (10, 1, 10),
    (10, 1, 10),
    ]
hyperpars["nested_cv"]["VKR"]["Chem"] = [
    (10, 10, 15),
    (10, 1, 15),
    (10, 1, 15),
    (10, 10, 15),
    (10, 10, 15),
    ]
hyperpars["nested_cv"]["VKR"]["DGI"] = [
    (10, 0.01, 15),
    (10, 0.1, 15),
    (10, 0.1, 15),
    (10, 0.1, 15),
    (10, 0.1, 15),
    ]
hyperpars["nested_cv"]["VKR"]["transporter"] = [
    (10, 0.1, 10),
    (10, 0.1, 15),
    (10, 0.1, 10),
    (10, 0.01, 10),
    (10, 0.01, 5),
    ]
hyperpars["nested_cv"]["VKR"]["pathway"] = [
    (10, 0.01, 15),
    (10, 0.01, 15),
    (10, 0.01, 15),
    (10, 0.01, 15),
    (10, 0.01, 15),
    ]
hyperpars["nested_cv"]["VKR"]["indication"] = [
    (10, 0.01, 15),
    (10, 0.1, 15),
    (10, 0.1, 15),
    (10, 0.01, 15),
    (10, 0.01, 15),
    ]
hyperpars["cv"]["VKR"] = {}
hyperpars["cv"]["VKR"]["target"] = (10, 100, 15)
hyperpars["cv"]["VKR"]["enzyme"] = (10, 100, 15)
hyperpars["cv"]["VKR"]["Chem"] =(100, 0.1, 15)
hyperpars["cv"]["VKR"]["DGI"] = (100, 0.01, 15)
hyperpars["cv"]["VKR"]["transporter"] = (100, 0.01, 15)
hyperpars["cv"]["VKR"]["pathway"] = (100, 100, 15)
hyperpars["cv"]["VKR"]["indication"] = (100, 100, 15)

In [15]:
hyperpars["nested_cv"]["LNSM_RLN"] = {}
hyperpars["nested_cv"]["LNSM_RLN"]["target"] = [
    (0.1, 1), 
    (0.1, 1), 
    (0.1, 1), 
    (0.1, 1), 
    (0.1, 1)
    ]
hyperpars["nested_cv"]["LNSM_RLN"]["enzyme"] = [
    (0.1, 10),
    (0.1, 10),
    (0.1, 10),
    (0.1, 100),
    (0.1, 100)
    ]
hyperpars["nested_cv"]["LNSM_RLN"]["Chem"] = [
    (0.9, 100),
    (0.9, 100),
    (0.4, 100),
    (0.4, 100),
    (0.4, 100)
    ]
hyperpars["nested_cv"]["LNSM_RLN"]["DGI"] = [
    (0.3, 100),
    (0.3, 100),
    (0.3, 100),
    (0.3, 100),
    (0.3, 100)
    ]
hyperpars["nested_cv"]["LNSM_RLN"]["transporter"] = [
    (0.1, 10),
    (0.2, 10),
    (0.1, 10),
    (0.1, 10),
    (0.1, 10)
    ]
hyperpars["nested_cv"]["LNSM_RLN"]["pathway"] = [
    (0.1, 10), 
    (0.1, 10), 
    (0.1, 10), 
    (0.1, 10), 
    (0.1, 10)
    ]
hyperpars["nested_cv"]["LNSM_RLN"]["indication"] = [
    (0.7, 100),
    (0.7, 100),
    (0.7, 100),
    (0.8, 100),
    (0.8, 100)
    ]
hyperpars["cv"]["LNSM_RLN"] = {}
hyperpars["cv"]["LNSM_RLN"]["target"] = (0.1, 10)
hyperpars["cv"]["LNSM_RLN"]["enzyme"] = (0.1, 100)
hyperpars["cv"]["LNSM_RLN"]["Chem"] = (0.8, 100)
hyperpars["cv"]["LNSM_RLN"]["DGI"] = (0.2, 100)
hyperpars["cv"]["LNSM_RLN"]["transporter"] = (0.1, 10)
hyperpars["cv"]["LNSM_RLN"]["pathway"] = (0.1, 100)
hyperpars["cv"]["LNSM_RLN"]["indication"] = (0.5, 100)

In [16]:
hyperpars["nested_cv"]["LNSM_jaccard"] = {}
hyperpars["nested_cv"]["LNSM_jaccard"]["target"] = [
    (0.1,), 
    (0.2,), 
    (0.1,), 
    (0.1,), 
    (0.3,)
    ]
hyperpars["nested_cv"]["LNSM_jaccard"]["enzyme"] = [
    (0.8,), 
    (0.4,), 
    (0.7,), 
    (0.1,), 
    (0.9,)
    ]
hyperpars["nested_cv"]["LNSM_jaccard"]["Chem"] = [
    (0.1,), 
    (0.8,), 
    (0.3,), 
    (0.4,), 
    (0.8,)
    ]
hyperpars["nested_cv"]["LNSM_jaccard"]["DGI"] = [
    (0.2,), 
    (0.1,), 
    (0.1,), 
    (0.1,), 
    (0.1,)
    ]
hyperpars["nested_cv"]["LNSM_jaccard"]["transporter"] = [
    (0.7,),
    (0.3,),
    (0.3,),
    (0.6,),
    (0.4,)
    ]
hyperpars["nested_cv"]["LNSM_jaccard"]["pathway"] = [
    (0.1,), 
    (0.1,), 
    (0.1,), 
    (0.1,), 
    (0.1,)
    ]
hyperpars["nested_cv"]["LNSM_jaccard"]["indication"] = [
    (0.1,), 
    (0.1,), 
    (0.1,), 
    (0.1,), 
    (0.1,)
    ]
hyperpars["cv"]["LNSM_jaccard"] = {}
hyperpars["cv"]["LNSM_jaccard"]["target"] = (0.1,)
hyperpars["cv"]["LNSM_jaccard"]["enzyme"] = (0.4,)
hyperpars["cv"]["LNSM_jaccard"]["Chem"] = (0.8,)
hyperpars["cv"]["LNSM_jaccard"]["DGI"] = (0.1,)
hyperpars["cv"]["LNSM_jaccard"]["transporter"] = (0.4,)
hyperpars["cv"]["LNSM_jaccard"]["pathway"] = (0.1,)
hyperpars["cv"]["LNSM_jaccard"]["indication"] = (0.4,)

In [17]:

# # Open and read the JSON file
# with open(f'results/hyperpars_{SEs_name}.xml', 'r') as xml_file:
#     hyperpars = json.load(xml_file)

In [18]:
for method in methods:
    # # method = "SKR"
    validation = "nested_cv"
    hyperparsOut[validation][method] = {}
    results[validation][method] = {}
    for str in features_names:
        print(f"using feature {str}")
        hyperparList = loadHyperpar(*all_hyperparlist[method],method_option=method)
        results[validation][method][str], hyperparsOut[validation][method][str] = evaluation(Y=SEs[SEs_name], X=features_dict[str], method_option=method,tuning_metrice=metrice,hyperparList=hyperparList,hyperparfixed=hyperpars[validation][method][str],Validation=validation,n_jobs=1)

    # method = "SKR"
    validation = "cv"
    hyperparsOut[validation][method] = {}
    results[validation][method] = {}
    for str in features_names:
        print(f"using feature {str}")
        hyperparList = loadHyperpar(*all_hyperparlist[method],method_option=method)
        results[validation][method][str], hyperparsOut[validation][method][str] = evaluation(Y=SEs[SEs_name], X=features_dict[str], method_option=method,tuning_metrice=metrice,hyperparList=hyperparList,hyperparfixed=hyperpars[validation][method][str],Validation=validation,n_jobs=1)

using feature target
The SKR requires hyperparameter lambda, c, sigma_X, sigma_Y
---------- nested cv start ----------
Fold: 0
number of hyperpars combination:  45
first few training idx:  [  55   75  219  255  516  563  569  804 1031 1046]
first few testing idx:  [ 260  324  368  553  601  692  712  998 1095 1115]
--- tuning end ---
target size: 226
------ best hyper pars:  (0.01, 0.6, 10, 100) ------
SKR starts:
SKR ends:
-----------
AUPRperdrug: 0.6051062970060217
AUROCperdrug: 0.946256519690964
AUPR+AUROCperdrug: 1.5513628166969857
AUPR: 0.4702759030019825
AUROC: 0.8419127337490575
AUPR+AUROC: 1.31218863675104
-----------
Fold: 1
number of hyperpars combination:  45
first few training idx:  [ 260  324  368  553  601  692  712  998 1095 1115]
first few testing idx:  [  55   75  219  255  516  563  569  804 1031 1046]
--- tuning end ---
target size: 226
------ best hyper pars:  (0.01, 0.6, 10, 100) ------
SKR starts:
SKR ends:
-----------
AUPRperdrug: 0.6317439735330859
AUROCperdrug:

/home/yezhao/anaconda3/envs/tf-2/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1090: ConvergenceWarning: Maximum number of iterations 1000 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


VKR ends:
-----------
AUPRperdrug: 0.6078036806287928
AUROCperdrug: 0.9472844392650953
AUPR+AUROCperdrug: 1.5550881198938882
AUPR: 0.45891078928961593
AUROC: 0.8412037793576326
AUPR+AUROC: 1.3001145686472486
-----------
Fold: 1
number of hyperpars combination:  75
first few training idx:  [ 260  324  368  553  601  692  712  998 1095 1115]
first few testing idx:  [  55   75  219  255  516  563  569  804 1031 1046]
--- tuning end ---
target size: 226
------ best hyper pars:  (10, 0.01, 15) ------
VKR starts:


/home/yezhao/anaconda3/envs/tf-2/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1090: ConvergenceWarning: Maximum number of iterations 1000 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


VKR ends:
-----------
AUPRperdrug: 0.6364473411577233
AUROCperdrug: 0.9480961734060978
AUPR+AUROCperdrug: 1.584543514563821
AUPR: 0.5138423241832403
AUROC: 0.8634511633169982
AUPR+AUROC: 1.3772934875002385
-----------
Fold: 2
number of hyperpars combination:  75
first few training idx:  [ 260  324  368  553  601  692  712  998 1095 1115]
first few testing idx:  [ 17  27  70 395 511 590 594 742 930 949]
--- tuning end ---
target size: 226
------ best hyper pars:  (10, 0.1, 15) ------
VKR starts:


...

VKR ends:
-----------
AUPRperdrug: 0.711283979282173
AUROCperdrug: 0.9392975636955443
AUPR+AUROCperdrug: 1.6505815429777173
AUPR: 0.575755941547425
AUROC: 0.8451289532959992
AUPR+AUROC: 1.4208848948434243
-----------
using feature pathway
The VKR requires hyperparameter lambda, sigma_X, k
---------- cv start ----------
--- tuning end ---
target size: 261
------ best hyper pars:  (100, 100, 15) ------
VKR starts:


/home/yezhao/anaconda3/envs/tf-2/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1090: ConvergenceWarning: Maximum number of iterations 1000 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


VKR ends:
-----------
AUPRperdrug: 0.6389911382877245
AUROCperdrug: 0.9435475468182835
AUPR+AUROCperdrug: 1.582538685106008
AUPR: 0.484616654304196
AUROC: 0.8277962413396083
AUPR+AUROC: 1.3124128956438041
-----------
using feature indication
The VKR requires hyperparameter lambda, sigma_X, k
---------- cv start ----------
--- tuning end ---
target size: 251
------ best hyper pars:  (100, 100, 15) ------
VKR starts:


/home/yezhao/anaconda3/envs/tf-2/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1090: ConvergenceWarning: Maximum number of iterations 1000 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


VKR ends:
-----------
AUPRperdrug: 0.6858423650106555
AUROCperdrug: 0.9425541648366716
AUPR+AUROCperdrug: 1.6283965298473273
AUPR: 0.5363853651698793
AUROC: 0.8431813061390275
AUPR+AUROC: 1.3795666713089068
-----------
using feature target
The Naive requires no hyperparameter
---------- nested cv start ----------
Fold: 0
number of hyperpars combination:  1
first few training idx:  [  55   75  219  255  516  563  569  804 1031 1046]
first few testing idx:  [ 260  324  368  553  601  692  712  998 1095 1115]
--- tuning end ---
target size: 226
------ best hyper pars:  () ------
Naive starts:
Naive ends:
-----------
AUPRperdrug: 0.603300106776754
AUROCperdrug: 0.9450909678114552
AUPR+AUROCperdrug: 1.5483910745882092
AUPR: 0.4577214870963392
AUROC: 0.83876886843943
AUPR+AUROC: 1.2964903555357692
-----------
Fold: 1
number of hyperpars combination:  1
first few training idx:  [ 260  324  368  553  601  692  712  998 1095 1115]
first few testing idx:  [  55   75  219  255  516  563  569  804

In [19]:
with open(f'results/hyperpars_{SEs_name}.xml', 'w') as xml_file:
   json.dump(hyperparsOut, xml_file)
with open(f'results/results_{SEs_name}_{filter}.xml', 'w') as xml_file:
   json.dump(results, xml_file)

In [20]:
with open(f'results/results_{SEs_name}_{filter}.xml', 'r') as xml_file:
    results = json.load(xml_file)

In [21]:
df = pd.DataFrame()
for m, fs in results["nested_cv"].items():
    for f, mes in fs.items():
        for me, scores in mes.items():
            temp_df = pd.DataFrame({
                'method': m,
                'feature': f,
                'metric': me,
                "score": scores
            })
            df = pd.concat([df, temp_df], ignore_index=True)

custom_order = ["pathway","Chem", "DGI",  "indication", "target", "transporter", "enzyme"]
df['feature'] = pd.Categorical(df['feature'], categories=custom_order, ordered=True)
df['method'] = pd.Categorical(df['method'], categories=methods, ordered=True)
df['metric'] = pd.Categorical(df['metric'], categories=metrice_names, ordered=True)
df2 = pd.pivot_table(df, values=['score'], index=["feature", "method"], aggfunc={'score': ["mean","std"]}, columns=["metric"])
df3 = df2.sort_index(axis=1, level='metric').sort_index(level='feature')
df3.to_excel(f'results/nested_cv_results_{SEs_name}_{filter}.xlsx')
df3

score                               \
                               mean        std              mean   
metric                   AUPR+AUROC AUPR+AUROC AUPR+AUROCperdrug   
feature     method                                                 
pathway     SKR            1.401852   0.020337          1.597627   
            KRR            1.387727   0.032016          1.588558   
            VKR            1.407823   0.025471          1.605898   
            Naive          1.347490   0.016012          1.592762   
            LNSM_RLN       0.298692   0.023203          0.198455   
            LNSM_jaccard   0.619868   0.032380          0.684587   
Chem        SKR            1.421332   0.037031          1.639458   
            KRR            1.418213   0.034645          1.640046   
            VKR            1.421951   0.035786          1.645576   
            Naive          1.418321   0.037221          1.640358   
            LNSM_RLN       1.099305   0.050358          1.225142   
            LNSM_jaccard   0.654024   0.043769          0.645131   
DGI         SKR            1.445483   0.031700          1.540142   
            KRR            1.444430   0.032197          1.533964   
            VKR            1.466495   0.031101          1.558442   
            Naive          1.298883   0.027640          1.552351   
            LNSM_RLN       1.189502   0.054936          1.225899   
            LNSM_jaccard   0.757225   0.127518          0.765136   
indication  SKR            1.537678   0.022922          1.629254   
            KRR            1.535037   0.024133          1.622366   
            VKR            1.559837   0.018383          1.643388   
            Naive          1.402369   0.018601          1.631277   
            LNSM_RLN       1.241171   0.068082          1.305068   
            LNSM_jaccard   0.322584   0.022181          0.201150   
target      SKR            1.330869   0.027861          1.566816   
            KRR            1.319941   0.026163          1.567990   
            VKR            1.327120   0.039160          1.571441   
            Naive          1.317984   0.021955          1.565123   
            LNSM_RLN       0.492521   0.047848          0.570147   
            LNSM_jaccard   0.699609   0.018020          0.757408   
transporter SKR            1.426760   0.013904          1.624576   
            KRR            1.406831   0.028218          1.612127   
            VKR            1.426931   0.025169          1.629309   
            Naive          1.392886   0.012380          1.630099   
            LNSM_RLN       1.311972   0.046806          1.519921   
            LNSM_jaccard   0.656619   0.086085          0.616331   
enzyme      SKR            1.373793   0.012777          1.587928   
            KRR            1.385651   0.025197          1.585035   
            VKR            1.388062   0.028752          1.590941   
            Naive          1.352309   0.019602          1.592145   
            LNSM_RLN       0.727474   0.469470          0.838754   
            LNSM_jaccard   0.608413   0.061231          0.573797   

                                                                          \
                                       std      mean       std      mean   
metric                   AUPR+AUROCperdrug     AUROC     AUROC      AUPR   
feature     method                                                         
pathway     SKR                   0.009471  0.859692  0.008044  0.542160   
            KRR                   0.019083  0.853503  0.014285  0.534224   
            VKR                   0.011186  0.862876  0.010578  0.544947   
            Naive                 0.009759  0.842988  0.006034  0.504502   
            LNSM_RLN              0.006651  0.202274  0.020063  0.096419   
            LNSM_jaccard          0.064551  0.426971  0.026506  0.192897   
Chem        SKR                   0.013021  0.853216  0.013850  0.568117   
            KRR                   0.012598  0.853147  0.013256  0

In [22]:
df = pd.DataFrame()
for m, fs in results["cv"].items():
    for f, mes in fs.items():
        for me, scores in mes.items():
            temp_df = pd.DataFrame({
                'method': m,
                'feature': f,
                'metric': me,
                "score": scores
            },index=["1"])
            df = pd.concat([df, temp_df], ignore_index=True)
df['feature'] = pd.Categorical(df['feature'], categories=custom_order, ordered=True)
df['method'] = pd.Categorical(df['method'], categories=methods, ordered=True)
df['metric'] = pd.Categorical(df['metric'], categories=metrice_names, ordered=True)
df2 = pd.pivot_table(df, values=['score'], index=["feature", "method"], columns="metric")
df2.to_excel(f'results/cv_results_{SEs_name}_{filter}.xlsx')
df2

score                                        \
metric                   AUPR+AUROC AUPR+AUROCperdrug     AUROC      AUPR   
feature     method                                                          
pathway     SKR            1.310736          1.583131  0.827500  0.483237   
            KRR            1.312279          1.583075  0.827761  0.484518   
            VKR            1.312413          1.582539  0.827796  0.484617   
            Naive          1.312752          1.583017  0.827950  0.484802   
            LNSM_RLN       1.250391          1.519825  0.806519  0.443872   
            LNSM_jaccard   0.596689          0.552211  0.424001  0.172688   
Chem        SKR            1.390528          1.630376  0.847032  0.543496   
            KRR            1.386339          1.628735  0.845250  0.541090   
            VKR            1.385926          1.628011  0.845009  0.540917   
            Naive          1.384795          1.627231  0.844561  0.540234   
            LNSM_RLN       1.001532          1.184612  0.661712  0.339820   
            LNSM_jaccard   0.688074          0.671081  0.492481  0.195592   
DGI         SKR            1.329337          1.556870  0.855696  0.473641   
            KRR            1.305328          1.552595  0.843558  0.461770   
            VKR            1.358205          1.554242  0.858602  0.499603   
            Naive          1.286939          1.551829  0.837310  0.449630   
            LNSM_RLN       1.196146          1.245890  0.753804  0.442342   
            LNSM_jaccard   0.542068          0.613420  0.409782  0.132287   
indication  SKR            1.413628          1.636252  0.859311  0.554317   
            KRR            1.381607          1.629187  0.844112  0.537495   
            VKR            1.379567          1.628397  0.843181  0.536385   
            Naive          1.381197          1.628857  0.843628  0.537568   
            LNSM_RLN       1.231834          1.321787  0.718807  0.513027   
            LNSM_jaccard   0.581554          0.768761  0.435962  0.145593   
target      SKR            1.304036          1.572386  0.839099  0.464936   
            KRR            1.304258          1.572474  0.839168  0.465090   
            VKR            1.303495          1.570392  0.838875  0.464620   
            Naive          1.310378          1.571122  0.840278  0.470101   
            LNSM_RLN       1.081186          1.121629  0.708843  0.372343   
            LNSM_jaccard   0.701423          0.821997  0.501604  0.199818   
transporter SKR            1.427749          1.651630  0.847429  0.580320   
            KRR            1.420889          1.651276  0.845164  0.575725   
            VKR            1.420885          1.650582  0.845129  0.575756   
            Naive          1.416597          1.650817  0.843730  0.572867   
            LNSM_RLN       1.273772          1.549407  0.792344  0.481428   
            LNSM_jaccard   0.706208          0.758838  0.444159  0.262049   
enzyme      SKR            1.366189          1.605825  0.845920  0.520269   
            KRR            1.373068          1.605624  0.848335  0.524734   
            VKR            1.372944          1.605188  0.848331  0.524613   
            Naive          1.355101          1.605271  0.841120  0.513980   
            LNSM_RLN       1.251677          1.487397  0.784637  0.467040   
            LNSM_jaccard   0.606533          0.576319  0.433489  0.173044   

                                                   
metric                   AUROCperdrug AUPRperdrug  
feature     method                                 
pathway     SKR              0.943660    0.639471  
            KRR              0.943613    0.639462  
            VKR              0.943548    0.638991  
            Naive            0.943610    0.639407  
            LNSM_RLN         0.914755    0.605070  
            LNSM_jaccard     0.343132    0.209080  
Chem        SKR              0.942571    0.687805  
            KRR              0.942066    0.6

In [23]:
df = pd.DataFrame()
for m, fs in results["nested_cv"].items():
    for f, mes in fs.items():
        for me, scores in mes.items():
            temp_df = pd.DataFrame({
                'method': m,
                'feature': f,
                'metric': me,
            }, index=["1"])
            temp_df2 = pd.concat([temp_df, pd.DataFrame(scores, columns=["1"]).T], axis=1)
            df = pd.concat([df, temp_df2], ignore_index=True)
for m in metrice_names:
    for f in features_names:
        df2 = df[(df["metric"] == m) & (df["feature"] == f)]
        df3 = df2.iloc[:, np.array([0, 3, 4, 5, 6, 7])]
        df4 = df3.set_index(df3.columns[0])
        df5 = df4.T.ptests(paired=True, stars=False)
        df5.to_excel(f'results/pvalue_{SEs_name}_{filter}_{f}_{m}.xlsx')